In [1]:
!pip install datasets
!pip install evaluate
!pip install tokenizers
!pip install transformers
!pip install bs4
!pip install lxml

In [2]:
import utils as utils
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
import os
from datasets import Dataset
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback
import numpy as np
import evaluate
from transformers import create_optimizer, AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# dataset_path = 'dataset'
# papers_path = 'papers'
# presentations_path = 'presentations'
#
# utils.move_xml_files(dataset_path, papers_path, presentations_path)

In [4]:
sample_xml_pres_path = "sample_data/presentations/slide.clean_tika.xml"
sample_xml_pres = utils.read_file(sample_xml_pres_path)
sample_xml_pres

sample_xml_paper_path = "sample_data/papers/Paper_BRM.tei.xml"
sample_xml_paper = utils.read_file(sample_xml_paper_path)
sample_xml_paper

'<?xml version="1.0" encoding="UTF-8"?>\n<TEI xmlns="http://www.tei-c.org/ns/1.0" \nxmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" \nxsi:schemaLocation="http://www.tei-c.org/ns/1.0 /Users/atharsefid/Desktop/grobid-0.5.3/grobid-home/schemas/xsd/Grobid.xsd"\n xmlns:xlink="http://www.w3.org/1999/xlink">\n\t<teiHeader xml:lang="en">\n\t\t<encodingDesc>\n\t\t\t<appInfo>\n\t\t\t\t<application version="0.5.3" ident="GROBID" when="2019-03-26T16:26+0000">\n\t\t\t\t\t<ref target="https://github.com/kermitt2/grobid">GROBID - A machine learning software for extracting information from scholarly documents</ref>\n\t\t\t\t</application>\n\t\t\t</appInfo>\n\t\t</encodingDesc>\n\t\t<fileDesc>\n\t\t\t<titleStmt>\n\t\t\t\t<title level="a" type="main">Best-Response Mechanisms</title>\n\t\t\t</titleStmt>\n\t\t\t<publicationStmt>\n\t\t\t\t<publisher/>\n\t\t\t\t<availability status="unknown"><licence/></availability>\n\t\t\t</publicationStmt>\n\t\t\t<sourceDesc>\n\t\t\t\t<biblStruct>\n\t\t\t\t\t<analy

In [5]:
sample_pres_text = utils.parse_presentation_xml(sample_xml_pres)
sample_pres_text[0]

'Noam Nisan, Michael Schapira, Gregory Valiant, and Aviv Zohar'

In [6]:
sample_paper_text = utils.parse_paper_xml(sample_xml_paper)
sample_paper_text[0]

'The basic object of study in game theory and in economics is the equilibrium: a "stable" state from which none of the players wish to deviate. Equilibrium is a static concept that often abstracts away the question of how it is reached. Once we start looking at dynamics, or at algorithms for finding equilibria, we cannot escape questions of the form "How is an equilibrium reached?". While there can be different formalizations of this question, in most cases, a truly satisfactory answer would have each player performing only simple "locally rational" actions and yet, mysteriously, the system would reach a global equilibrium. The simplest example of such phenomena is repeated best-response dynamics: each player selects the best (locally optimal) response to what others are currently doing, and this process goes on "for a while" until it "converges" to what must be a (pure Nash) equilibrium. Convergence of repeated bestresponse is, unfortunately, not guaranteed in general, and is the subj

In [7]:
sample_paper_title = utils.parse_title(sample_xml_paper)
print(sample_paper_title)

Best-Response Mechanisms


In [8]:
sample_pres_preprocessed = utils.preprocess_text(sample_pres_text)

In [9]:
sample_paper_preprocessed = utils.preprocess_text(sample_paper_text)

In [10]:
print(sample_pres_preprocessed[0])
print(sample_paper_preprocessed[0])

['noam', 'nisan', 'michael', 'schapira', 'gregori', 'valiant', 'aviv', 'zohar']
['basic', 'object', 'studi', 'game', 'theori', 'econom', 'equilibrium', 'stabl', 'state', 'none', 'player', 'wish', 'deviat', 'equilibrium', 'static', 'concept', 'often', 'abstract', 'away', 'question', 'reach', 'start', 'look', 'dynam', 'algorithm', 'find', 'equilibria', 'escap', 'question', 'form', 'equilibrium', 'reach', 'differ', 'formal', 'question', 'case', 'truli', 'satisfactori', 'answer', 'would', 'player', 'perform', 'simpl', 'local', 'ration', 'action', 'yet', 'mysteri', 'system', 'would', 'reach', 'global', 'equilibrium', 'simplest', 'exampl', 'phenomena', 'repeat', 'bestrespons', 'dynam', 'player', 'select', 'best', 'local', 'optim', 'respons', 'other', 'current', 'process', 'goe', 'converg', 'must', 'pure', 'nash', 'equilibrium', 'converg', 'repeat', 'bestrespons', 'unfortun', 'guarante', 'gener', 'subject', 'much', 'research', 'converg', 'sophist', 'locallyr', 'dynam', 'eg', 'fictiti', 'play'

In [11]:
presentation_word_model = Word2Vec(sentences = sample_pres_preprocessed, vector_size = 50, window = 5, min_count = 1, workers = 3, sg = 1)
paper_word_model = Word2Vec(sentences = sample_paper_preprocessed, vector_size = 50, window = 5, min_count = 1, workers = 3, sg = 1)

In [12]:
print(presentation_word_model)
print(paper_word_model)

Word2Vec<vocab=187, vector_size=50, alpha=0.025>
Word2Vec<vocab=829, vector_size=50, alpha=0.025>


In [13]:
EMBEDDING_PRES_MODEL_FILE = "pres_word_model.txt"
EMBEDDING_PAPER_MODEL_FILE = "paper_word_model.txt"

presentation_word_model.wv.save_word2vec_format(EMBEDDING_PRES_MODEL_FILE, binary=False)
paper_word_model.wv.save_word2vec_format(EMBEDDING_PAPER_MODEL_FILE, binary=False)

In [14]:
presentation_folder = "sample_data/presentations"   #Original: "dataset/presentations"
paper_folder = "sample_data/papers"                 #Original: "dataset/papers"

papers_data = []
presentations_data = []
id2label = {}
label2id = {}
unknowns = 0

# Loop through presentation XML files
for presentation_file in os.listdir(presentation_folder):
    file_path = os.path.join(presentation_folder, presentation_file)
    if os.path.isfile(file_path):
      file_content = utils.read_file(file_path)
      if file_content:
          # Parse presentation XML
          presentation_data = utils.parse_presentation_xml(file_content)
          # Preprocess presentation data
          # preprocessed_presentation_data = utils.preprocess_text(presentation_data)
          # presentations_data.append(preprocessed_presentation_data)
          presentations_data.append(presentation_data)

# Loop through paper XML files
for idx, paper_file in enumerate(os.listdir(paper_folder)):
    file_path = os.path.join(paper_folder, paper_file)
    file_content = utils.read_file(file_path)
    if file_content:
        # Parse paper XML
        paper_data = utils.parse_paper_xml(file_content)
        # Preprocess paper data
        # preprocessed_paper_data = utils.preprocess_text(paper_data)
        title = utils.parse_title(file_content)
        if title is not None:
            # Check if title is already in label2id
            if title not in label2id:
                # If title is not in label2id, add it directly
                id2label[idx] = title
                label2id[title] = idx
            else:
                # If title is already in label2id, generate a unique title
                unique_title = f"{title}_{idx}"
                id2label[idx] = unique_title
                label2id[unique_title] = idx
            # Append paper data
        else:
            unknowns += 1  # Increment unknowns counter
            unique_title = f"unknown_{idx}"
            id2label[idx] = unique_title
            label2id[unique_title] = idx
        # papers_data.append(preprocessed_paper_data)
        papers_data.append(paper_data)
data = {
    "papers": papers_data,
    "presentations": presentations_data
}
# presentation_to_paper = utils.create_presentation_to_paper_mapping(presentation_folder, paper_folder)
#
# presentations_data = utils.process_presentation_folder(presentation_folder)
# papers_data = utils.process_papers_folder(paper_folder)



In [15]:
# print(presentations_data[0])
# print(papers_data[3])
# print(data)
for idx in range(5):
    paper = data["papers"][idx][0] if idx < len(data["papers"]) and data["papers"][idx] else "N/A"
    presentation = data["presentations"][idx][0] if idx < len(data["presentations"]) and data["presentations"][idx] else "N/A"
    print(f"Pair {idx+1}:")
    print("Paper:", paper)
    print("Presentation:", presentation)
    print()

Pair 1:
Paper: Text simplification (TS) is the task of modifying an original text into a simpler version of it. One of the main parameters for defining a suitable simplification is the target audience. Examples include elderly, children, cognitively impaired users, nonnative speakers and low-literacy readers.Traditionally, work on TS has been divided in lexical simplification (LS) and syntactic simplification (SS). LS (Paetzold, 2016) deals with the identification and replacement of complex words or phrases. SS (Siddharthan, 2011) performs structural transformations such as changing a sentence from passive to active voice. However, most recent approaches learn transformations from corpora, addressing simplification at lexical and syntactic levels altogether. These include either learning tree-based transformations (Woodsend and La- pata, 2011;Paetzold and Specia, 2013) or using machine translation (MT)-based techniques ( Zhu et al., 2010;Coster and Kauchak, 2011a;Wubben et al., 2012;Na

In [16]:
labels = []
for key, value in label2id.items():
    labels.append(value)

In [17]:
# print(label2id)
# print(labels)
print(len(presentations_data))
print(len(papers_data))
print(len(label2id))
print(len(labels))
print(unknowns)

6
6
6
6
0


In [18]:
## distilBERT tokenizer to preprocess
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [74]:
## split into train and test sets with labels
presentation_data = utils.stringify(data["presentations"])
#print(data["presentations"])
#print(presentation_data)
#print(len(presentation_data))
X_train, X_test, y_train, y_test = train_test_split(data["presentations"], labels, test_size=0.2, train_size=0.8, random_state=42)

train_text = []
test_text = []
train_label = []
test_label = []

for text, label in zip(X_train, y_train):
    train_text.append(text)
    train_label.append(label)

for text, label in zip(X_test, y_test):
    test_text.append(text)
    test_label.append(label)
    
#print(train_text)

#train_text = utils.stringify(train_data["text"])
#print(train_text[0])

#test_text = utils.stringify(test_data["text"])
#print(test_text)
#print(len(test_text))
#print(len(test_label))

train_dict = {
    "label": train_label,
    "text": train_text
}

test_dict = {
    "label": test_label,
    "text": test_text
}

train_data = Dataset.from_dict(train_dict)
test_data = Dataset.from_dict(test_dict)

print(test_data[1])
print(train_data[1])


{'label': 1, 'text': ['Personalized Review Generation by Expanding Phrases and Attending on Aspect-Aware Representations Jianmo Ni and Julian McAuley UC San Diego July 2018', 'Reviews in recommender system', 'Help user write reviews in an easier way? Expand and rewrite phrases  Estimate reactions and provide suggestions', 'Incorporate information & knowledge User and item attribute  Dong et al. EACL 2017. Learning to Generate Product Reviews from Attributes.  Tang et al. Arxiv 2016. Context-aware Natural Language Generation with Recurrent Neural Networks. Rich auxiliary information!  Short phrases (user input)  Product title  Aspect preference', 'Incorporate information & knowledge Aspect preference  User-aspect preference  Item-aspect relation Aspects Representative words Service vendor seller supplier reply refund Price price value overall dependable reliable Screen screen touchscreen browse display scrolling Case case cover briefcase portfolio padded Drive drive disk copying copied 

In [75]:
## preprocessing function to apply tokenizer over whole dataset
def preprocess_function(data):
    return tokenizer(data["text"],padding="max_length", truncation=True)

In [76]:
## batch to process multiple at once for faster compute
#print(train_data)
#print(test_data)

#print(len(train_data["text"][0]))
#print(len(test_data["text"][0]))
#preprocessed_data = [preprocess_function(item) for item in train_data["text"][0]]
#tokenized_train_data = train_data.map(preprocess_function, batched=True)
#print(tokenized_train_data)

#tokenized_test_data = test_data.map(preprocess_function, batched=True)
#print(tokenized_test_data)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [77]:
## padding dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [78]:
accuracy = evaluate.load("accuracy")

In [79]:
## metrics function that passes preds and labels to compute metrics
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [80]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(train_data) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
# try 3e-5
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [66]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [82]:
print(train_data)
print(tokenized_train_data["label"])
print(tokenized_train_data["text"])
print(tokenized_train_data["input_ids"])
#print(tokenized_train_data["attention-mask"])
#print(train_data["attention-mask"])
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data,
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_test_data,
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator,
)
#print(tf_train_set)

Dataset({
    features: ['label', 'text'],
    num_rows: 4
})
[5, 2, 4, 3]
['Noam Nisan, Michael Schapira, Gregory Valiant, and Aviv Zohar Motivation Equilibrium is the basic object of study in game theory. Question: How is an equilibrium reached? In a truly satisfactory answer each players rule of behavior is simple and locally rational repeated best-response repeated better-response regret-minimization Motivation Repeated best-response is often employed in practice e.g., Internet routing We ask: When is such locallyrational behavior really rational? Repeated best-response is not always best. *the game is solvable through elimination of dominated strategies. Overview of Results We identify a small class of games for which: 1. Repeated best-response converges (quickly) from any initial point. While small, this class covers several important examples: Internet Routing, Cost Sharing, Stable Roommates, Congestion Control. The Setting The repeated best-response strategy: When a players tur

In [83]:
model.compile(optimizer=optimizer) #Transformer has default task-relevant loss function


In [84]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [85]:
!pip install huggingface_hub
import huggingface_hub
#huggingface_hub.login()

push_to_hub_callback = PushToHubCallback(
    output_dir="CS4120final",
    tokenizer=tokenizer,
)



C:\Users\chris\CS4120final\CS4120final is already a clone of https://huggingface.co/crisis101/CS4120final. Make sure you pull the latest changes with `repo.git_pull()`.


In [86]:
callbacks = [metric_callback, push_to_hub_callback]

In [87]:
print(len(tokenized_train_data["text"][3]))

3306


In [88]:
print(tf_train_set)
model.fit(tf_train_set)
#model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(1, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(1, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(1,), dtype=tf.int64, name=None))>


InvalidArgumentError: Graph execution error:

Detected at node tf_distil_bert_for_sequence_classification_1/distilbert/embeddings/Gather_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\asy

In [ ]:
# ## check if works
#
# word_tokenizer = Tokenizer()
# word_tokenizer.fit_on_texts(presentations_data)
# encoded = word_tokenizer.texts_to_sequences(presentations_data)
#
# char_tokenizer = Tokenizer()
# char_tokenizer.fit_on_texts(papers_data)
# encoded = char_tokenizer.texts_to_sequences(papers_data)

In [ ]:
# ## correct implementation for LR ?
#
# word_map, index_map = utils.read_embeddings("spooky_embedding_word.txt", word_tokenizer)
# char_map, char_index_map = utils.read_embeddings("spooky_embedding_char.txt", char_tokenizer)

In [ ]:
# tfidf_vectorizer = TfidfVectorizer()
# X = tfidf_vectorizer.fit_transform(presentations_data)
#
# y = list(range(len(presentations_data)))
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# logreg_model = LogisticRegression(max_iter=1000)
# logreg_model.fit(X_train, y_train)
#
# y_pred = logreg_model.predict(X_test)
#
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [ ]:
# # print(y_pred[0])
# # print(y_test[0])
# # print(presentations_data)
# print(X_train)
# print(X_test)
# # print(len(y))